# Use the built-in optimization algorithms

In this example, we will use the built-in optimization algorithms to minimize a built-in benchmark function

In [1]:
from f3dasm.design import make_nd_continuous_domain

domain = make_nd_continuous_domain(bounds=[[0., 1.], [0., 1.]])
domain

Invalid MIT-MAGIC-COOKIE-1 key

Domain(input_space={'x0': ContinuousParameter(lower_bound=0.0, upper_bound=1.0, log=False), 'x1': ContinuousParameter(lower_bound=0.0, upper_bound=1.0, log=False)}, output_space={})

We sample random points from the search space and evaluate the function at these points with the `'Ackley'` function:

In [11]:
from f3dasm import ExperimentData, create_datagenerator, create_sampler

# Initialize the sampler and the data generator
sampler = create_sampler(sampler='random', seed=42)
data_generator = create_datagenerator(data_generator='Ackley', scale_bounds=[[0., 1.], [0., 1.]], offset=False)

# Sample from the domain
experiment_data = ExperimentData(domain=domain)
experiment_data = sampler.call(experiment_data, n_samples=10)

# Evaluate the samples with the Ackley function
data_generator.arm(experiment_data)
experiment_data = data_generator.call(experiment_data)

experiment_data

jobs     input               output
                   x0        x1          y
0  finished  0.773956  0.438878  18.573435
1  finished  0.858598  0.697368  21.308662
2  finished  0.094177  0.975622  21.815402
3  finished  0.761140  0.786064  20.719672
4  finished  0.128114  0.450386  21.396762
5  finished  0.370798  0.926765  21.396784
6  finished  0.643865  0.822762  21.112180
7  finished  0.443414  0.227239  19.952145
8  finished  0.554585  0.063817  21.959833
9  finished  0.827631  0.631664  21.527180

Now we will use the built-in optimization algorithm `'Nelder-Mead'` to minimize the output paramater `'y'`

## Method 1: Using the Optimizer Name as a String

You can easily initiate optimization by calling the `optimize()` method of the `ExperimentData` object and passing the optimizer's name as a string.

- **Iterative Optimization Process**: Since optimization occurs iteratively, you need to evaluate new points and update the `ExperimentData` object with the resulting data. To facilitate this, pass a `DataGenerator` object as an argument to the `optimize()` method. Any additional arguments for the `DataGenerator` can be provided as a dictionary through the `kwargs` parameter.
- **Default and Custom Hyperparameters**: By default, the optimizer runs with predefined hyperparameters. To customize them, pass the desired hyperparameters as keyword arguments to the `optimize()` method.
- **Specifying Iterations**: Use the `iterations` keyword to specify the number of function evaluations to perform during optimization.
- **Initial Point Strategy (`x0_strategy`)**: This argument defines how the starting point of the optimization is selected:
  - `'best'` (default): Uses the best point in the `ExperimentData` object as the starting point.
  - `'last'`: Uses the most recently added point in the `ExperimentData` object as the starting point.
  - `'new'`: Samples a new random point from the search space. Optionally, you can specify a `sampler` argument to control how the initial point is sampled (defaults to random uniform sampling).

In [12]:
from f3dasm import create_optimizer

optimizer = create_optimizer(optimizer='randomsearch')

optimizer.arm(experiment_data)

for _i in range(10):
    experiment_data += optimizer.call(experiment_data)
    experiment_data = data_generator.call(experiment_data)


jobs     input               output
                    x0        x1          y
0   finished  0.773956  0.438878  18.573435
1   finished  0.858598  0.697368  21.308662
2   finished  0.094177  0.975622  21.815402
3   finished  0.761140  0.786064  20.719672
4   finished  0.128114  0.450386  21.396762
5   finished  0.370798  0.926765  21.396784
6   finished  0.643865  0.822762  21.112180
7   finished  0.443414  0.227239  19.952145
8   finished  0.554585  0.063817  21.959833
9   finished  0.827631  0.631664  21.527180
10  finished  0.180688  0.888125  21.541318
11  finished  0.292089  0.558842  19.060338
12  finished  0.816474  0.449831  20.829402
13  finished  0.595512  0.644202  18.089648
14  finished  0.807139  0.254388  20.065375
15  finished  0.618721  0.300948  18.606269
16  finished  0.988954  0.657496  21.245709
17  finished  0.456277  0.871846  21.105018
18  finished  0.171977  0.268508  21.464503
19  finished  0.419083  0.425991  14.339684

In [ ]:
experiment_data

## Method 2: Importing the optimizer from the `f3dasm.optimization` module

Alternatively, you can import the optimizer from the `f3dasm.optimization` module and use it directly. This method provides more flexibility in customizing the optimizer's hyperparameters and behavior.

In [13]:
# Recreate the experiment data before optimizing

# Initialize the sampler and the data generator
sampler = create_sampler(sampler='random', seed=42)
data_generator = create_datagenerator(data_generator='Ackley', scale_bounds=[[0., 1.], [0., 1.]], offset=False)

# Sample from the domain
experiment_data = ExperimentData(domain=domain)
experiment_data = sampler.call(experiment_data, n_samples=10)

# Evaluate the samples with the Ackley function
data_generator.arm(experiment_data)
experiment_data = data_generator.call(experiment_data)

experiment_data

jobs     input               output
                   x0        x1          y
0  finished  0.773956  0.438878  18.573435
1  finished  0.858598  0.697368  21.308662
2  finished  0.094177  0.975622  21.815402
3  finished  0.761140  0.786064  20.719672
4  finished  0.128114  0.450386  21.396762
5  finished  0.370798  0.926765  21.396784
6  finished  0.643865  0.822762  21.112180
7  finished  0.443414  0.227239  19.952145
8  finished  0.554585  0.063817  21.959833
9  finished  0.827631  0.631664  21.527180

In [14]:
from f3dasm.optimization import random_search

The `nelder_mead` function can be called without parameters to use the default hyperparameters. To customize the optimizer, pass the desired hyperparameters as keyword arguments to the constructor:

In [15]:
optimizer = random_search(seed=42)

Now, we call the `optimize()` method with the optimizer object to perform the optimization.

In [16]:
from f3dasm import create_optimizer

optimizer.arm(experiment_data)

for _i in range(10):
    experiment_data += optimizer.call(experiment_data)
    experiment_data = data_generator.call(experiment_data)

experiment_data

jobs     input               output
                    x0        x1          y
0   finished  0.773956  0.438878  18.573435
1   finished  0.858598  0.697368  21.308662
2   finished  0.094177  0.975622  21.815402
3   finished  0.761140  0.786064  20.719672
4   finished  0.128114  0.450386  21.396762
5   finished  0.370798  0.926765  21.396784
6   finished  0.643865  0.822762  21.112180
7   finished  0.443414  0.227239  19.952145
8   finished  0.554585  0.063817  21.959833
9   finished  0.827631  0.631664  21.527180
10  finished  0.773956  0.438878  18.573435
11  finished  0.858598  0.697368  21.308662
12  finished  0.094177  0.975622  21.815402
13  finished  0.761140  0.786064  20.719672
14  finished  0.128114  0.450386  21.396762
15  finished  0.370798  0.926765  21.396784
16  finished  0.643865  0.822762  21.112180
17  finished  0.443414  0.227239  19.952145
18  finished  0.554585  0.063817  21.959833
19  finished  0.827631  0.631664  21.527180